## Import Data from CSV

In [6]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [8]:
#if we sum all targets we will get the number of targets that are 1s
num_one_targets = int(np.sum(targets_all))

#set counter for 0 targets equal to 0
zero_targets_counter = 0

#variable to record indices to be removed
indices_to_remove = []

#itirate the dataset and balance it
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        #increase the 0s counter by 1, if target is 0
        zero_targets_counter += 1
        #if target at position i is 0, and number of 0s > than 1s, add to index
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

### Standardize the dataset

In [9]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [10]:
#Shuffle purpose is to keep the same info in random order
#Why shuffle? Because we will be batching the data.

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split dataset into train, validation, and test

In [19]:
samples_count = shuffled_inputs.shape[0]

#I'll use the 80-10-10 split for train, validation, and test
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

#extract the data
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1791.0 3579 0.5004191114836547
230.0 447 0.5145413870246085
216.0 448 0.48214285714285715


#### from the numbers above we can safely assume all three sets are balanced (close as 50% is what we're looking for)

In [20]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)